In [1]:
pip install ucimlrepo

In [2]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
from collections import Counter

def prepare_data():
    # Fetch and prepare the dataset
    credit_approval = fetch_ucirepo(id=27)
    df = pd.DataFrame(credit_approval.data.features)
    df = pd.concat([df, credit_approval.data.targets], axis=1)

    # Handle missing values
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].fillna(df[column].mode()[0])
        elif pd.api.types.is_numeric_dtype(df[column]):
            df[column] = df[column].fillna(df[column].mean())

    # Discretize numeric columns
    for column in df.columns:
        if df[column].dtype != 'object' and column != 'A16':  # Skip target variable
            try:
                df[column] = pd.qcut(df[column], q=3, labels=['low', 'medium', 'high'])
            except:
                print(f"Could not discretize column {column}")

    return df


In [3]:
def generate_all_possible_transitions(df):
    # Initialize an empty list to store the feature-value transitions
    feature_value_transitions = []

    # Iterate over each feature in the DataFrame (excluding the target variable 'A16')
    for feature in df.columns:
        if feature != 'A16':  # Skip the target variable
            # Get unique values for the current feature
            feature_values = df[feature].unique()

            # Generate all possible pairs of feature values (old_value, new_value)
            for old_value in feature_values:
                for new_value in feature_values:
                        transition = (feature, old_value, new_value)
                        feature_value_transitions.append(transition)

    return feature_value_transitions

In [4]:
df=prepare_data()


Could not discretize column A15
Could not discretize column A11


In [5]:
atomic_action_sets=generate_all_possible_transitions(df)
atomic_action_sets

[('A15', 0, 0),
 ('A15', 0, 560),
 ('A15', 0, 824),
 ('A15', 0, 3),
 ('A15', 0, 31285),
 ('A15', 0, 1349),
 ('A15', 0, 314),
 ('A15', 0, 1442),
 ('A15', 0, 200),
 ('A15', 0, 2690),
 ('A15', 0, 245),
 ('A15', 0, 1208),
 ('A15', 0, 1260),
 ('A15', 0, 11),
 ('A15', 0, 10000),
 ('A15', 0, 5000),
 ('A15', 0, 4000),
 ('A15', 0, 35),
 ('A15', 0, 713),
 ('A15', 0, 551),
 ('A15', 0, 500),
 ('A15', 0, 300),
 ('A15', 0, 221),
 ('A15', 0, 2283),
 ('A15', 0, 100),
 ('A15', 0, 15),
 ('A15', 0, 284),
 ('A15', 0, 1236),
 ('A15', 0, 5800),
 ('A15', 0, 730),
 ('A15', 0, 400),
 ('A15', 0, 50000),
 ('A15', 0, 456),
 ('A15', 0, 15108),
 ('A15', 0, 2954),
 ('A15', 0, 2),
 ('A15', 0, 20),
 ('A15', 0, 27),
 ('A15', 0, 225),
 ('A15', 0, 1),
 ('A15', 0, 38),
 ('A15', 0, 5),
 ('A15', 0, 130),
 ('A15', 0, 147),
 ('A15', 0, 210),
 ('A15', 0, 11202),
 ('A15', 0, 1332),
 ('A15', 0, 50),
 ('A15', 0, 258),
 ('A15', 0, 567),
 ('A15', 0, 1000),
 ('A15', 0, 2510),
 ('A15', 0, 809),
 ('A15', 0, 610),
 ('A15', 0, 150),
 ('

In [6]:
positive_df=df[df['A16']=='+'].drop(['A16'],axis=1)
negative_df=df[df['A16']=='-'].drop(['A16'],axis=1)

In [7]:
import collections
posi_freq_table=positive_df.apply(collections.Counter)
neg_freq_table=negative_df.apply(collections.Counter)

In [8]:
def create_action_table(positive_df, negative_df):
    # Initialize an empty list to store the action rules
    action_table = []

    # Compare each row in negative_df with every row in positive_df
    for _, neg_row in negative_df.iterrows():
        for _, pos_row in positive_df.iterrows():
            # Identify feature differences between negative and positive rows
            changes = []
            for feature in positive_df.columns:
                if neg_row[feature] != pos_row[feature]:
                    changes.append((feature, neg_row[feature], pos_row[feature]))  # (Feature, old_value, new_value)
            if changes:
                action_table.append(changes)  # Add changes for this comparison

    return action_table

In [9]:
support_atomic_actionsets={}
for (feat,lhs,rhs) in atomic_action_sets:
  support_atomic_actionsets[(feat,lhs,rhs)]=posi_freq_table[feat][lhs]*neg_freq_table[feat][rhs]


In [10]:
min_support=4000
best_atomic_actionsets=[]
for i,j in support_atomic_actionsets.items():
  if j>=min_support:
    best_atomic_actionsets.append((i,j))

In [11]:
best_atomic_actionsets=sorted(best_atomic_actionsets,key=lambda x:x[1],reverse=True)
best_atomic_actionsets

[(('A13', 'g', 'g'), 97006),
 (('A9', 't', 'f'), 86904),
 (('A5', 'g', 'g'), 68900),
 (('A4', 'u', 'u'), 68900),
 (('A10', 't', 'f'), 62073),
 (('A1', 'b', 'b'), 56639),
 (('A7', 'v', 'v'), 40655),
 (('A12', 'f', 'f'), 34293),
 (('A12', 't', 'f'), 31098),
 (('A5', 'g', 'p'), 30680),
 (('A4', 'u', 'y'), 30680),
 (('A11', 0, 0), 29106),
 (('A10', 'f', 'f'), 29106),
 (('A12', 'f', 't'), 27370),
 (('A8', 'high', 'low'), 26671),
 (('A1', 'a', 'b'), 26558),
 (('A12', 't', 't'), 24820),
 (('A1', 'b', 'a'), 23408),
 (('A14', 'low', 'medium'), 22320),
 (('A9', 't', 't'), 21868),
 (('A3', 'high', 'low'), 21509),
 (('A15', 0, 0), 20566),
 (('A7', 'h', 'v'), 20445),
 (('A8', 'high', 'medium'), 20115),
 (('A14', 'low', 'high'), 18720),
 (('A8', 'medium', 'low'), 18437),
 (('A3', 'high', 'medium'), 18221),
 (('A2', 'high', 'low'), 18034),
 (('A10', 't', 't'), 17974),
 (('A2', 'high', 'medium'), 17653),
 (('A14', 'high', 'medium'), 15345),
 (('A14', 'low', 'low'), 14112),
 (('A8', 'medium', 'medium')

In [12]:
action_table=create_action_table(positive_df,negative_df)
action_table

[[('A14', 'high', 'medium'),
  ('A13', 's', 'g'),
  ('A12', 't', 'f'),
  ('A11', 0, 1),
  ('A10', 'f', 't'),
  ('A7', 'bb', 'v'),
  ('A6', 'e', 'w'),
  ('A3', 'high', 'low')],
 [('A15', 0, 560),
  ('A14', 'high', 'low'),
  ('A13', 's', 'g'),
  ('A12', 't', 'f'),
  ('A11', 0, 6),
  ('A10', 'f', 't'),
  ('A8', 'medium', 'high'),
  ('A7', 'bb', 'h'),
  ('A6', 'e', 'q'),
  ('A3', 'high', 'medium'),
  ('A2', 'medium', 'high'),
  ('A1', 'b', 'a')],
 [('A15', 0, 824),
  ('A13', 's', 'g'),
  ('A12', 't', 'f'),
  ('A7', 'bb', 'h'),
  ('A6', 'e', 'q'),
  ('A3', 'high', 'low'),
  ('A1', 'b', 'a')],
 [('A15', 0, 3),
  ('A14', 'high', 'low'),
  ('A13', 's', 'g'),
  ('A11', 0, 5),
  ('A10', 'f', 't'),
  ('A8', 'medium', 'high'),
  ('A7', 'bb', 'v'),
  ('A6', 'e', 'w'),
  ('A3', 'high', 'medium')],
 [('A14', 'high', 'medium'),
  ('A12', 't', 'f'),
  ('A7', 'bb', 'v'),
  ('A6', 'e', 'w'),
  ('A2', 'medium', 'low')],
 [('A13', 's', 'g'),
  ('A8', 'medium', 'high'),
  ('A7', 'bb', 'v'),
  ('A6', 'e', 'm

In [13]:
def prune_and_reorder_action_table(action_table, best_atomic_actionsets):
    """
    Prunes entries from action table that don't meet minimum support and
    reorders remaining entries by descending support.

    Args:
        action_table: List of lists containing action sets
        best_atomic_actionsets: Set of tuples ((feature, old_val, new_val), support)
                               that meet minimum support threshold

    Returns:
        List of pruned and reordered action sets
    """
    # Convert best_atomic_actionsets to a dict for easier lookup of support values
    support_dict = {action[0]: action[1] for action in best_atomic_actionsets}

    pruned_table = []

    # Iterate through each row in action table
    for row in action_table:
        # Check if all actions in the row are in best_atomic_actionsets
        valid_actions = []
        for action in row:
            if action in support_dict:
                # Store tuple of (action, support) for sorting
                valid_actions.append((action, support_dict[action]))

        # If we have valid actions after pruning
        if valid_actions:
            # Sort actions by support value in descending order
            sorted_actions = sorted(valid_actions, key=lambda x: x[1], reverse=True)
            # Keep only the actions, discard support values
            pruned_table.append([action for action, _ in sorted_actions])

    return pruned_table

# Prune and reorder the action table
pruned_action_table = prune_and_reorder_action_table(action_table, best_atomic_actionsets)

# Print first few rows of pruned and reordered action table
print("\nFirst few rows of pruned and reordered action table:")
for i, row in enumerate(pruned_action_table[:5]):
    print(f"Row {i + 1}: {row}")


First few rows of pruned and reordered action table:
Row 1: [('A12', 't', 'f'), ('A3', 'high', 'low'), ('A14', 'high', 'medium'), ('A10', 'f', 't'), ('A7', 'bb', 'v'), ('A13', 's', 'g')]
Row 2: [('A12', 't', 'f'), ('A1', 'b', 'a'), ('A3', 'high', 'medium'), ('A14', 'high', 'low'), ('A2', 'medium', 'high'), ('A10', 'f', 't'), ('A8', 'medium', 'high'), ('A13', 's', 'g')]
Row 3: [('A12', 't', 'f'), ('A1', 'b', 'a'), ('A3', 'high', 'low'), ('A13', 's', 'g')]
Row 4: [('A3', 'high', 'medium'), ('A14', 'high', 'low'), ('A10', 'f', 't'), ('A8', 'medium', 'high'), ('A7', 'bb', 'v'), ('A13', 's', 'g')]
Row 5: [('A12', 't', 'f'), ('A14', 'high', 'medium'), ('A2', 'medium', 'low'), ('A7', 'bb', 'v')]


In [14]:
for i in pruned_action_table:
  print(i)

Streaming output truncated to the last 5000 lines.
[('A7', 'h', 'v'), ('A14', 'high', 'medium'), ('A3', 'medium', 'low'), ('A2', 'medium', 'low'), ('A5', 'p', 'g'), ('A4', 'y', 'u')]
[('A12', 'f', 't'), ('A1', 'b', 'a'), ('A3', 'medium', 'low'), ('A2', 'medium', 'low'), ('A13', 'g', 's'), ('A8', 'medium', 'high'), ('A7', 'h', 'ff')]
[('A1', 'b', 'a'), ('A7', 'h', 'v'), ('A3', 'medium', 'low'), ('A5', 'p', 'g'), ('A4', 'y', 'u')]
[('A7', 'h', 'v'), ('A8', 'medium', 'low'), ('A3', 'medium', 'low'), ('A2', 'medium', 'high'), ('A10', 'f', 't')]
[('A1', 'b', 'a'), ('A3', 'medium', 'low'), ('A5', 'p', 'g'), ('A4', 'y', 'u'), ('A2', 'medium', 'high'), ('A10', 'f', 't'), ('A8', 'medium', 'high')]
[('A12', 'f', 't'), ('A7', 'h', 'v'), ('A3', 'medium', 'low'), ('A5', 'p', 'g'), ('A4', 'y', 'u'), ('A10', 'f', 't'), ('A8', 'medium', 'high')]
[('A7', 'h', 'v'), ('A5', 'p', 'g'), ('A4', 'y', 'u'), ('A2', 'medium', 'high'), ('A10', 'f', 't'), ('A3', 'medium', 'high'), ('A8', 'medium', 'high')]
[('A1'

In [15]:
class FPNode:
    def __init__(self, action=None, count=1, parent=None):
        """Initialize a node in the FP-tree."""
        self.action = action  # (feature, old_value, new_value)
        self.count = count
        self.parent = parent
        self.children = {}
        self.next = None  # Link to next node with same action (for header table)

class FPTree:
    def __init__(self):
        """Initialize the FP-tree structure."""
        self.root = FPNode()  # Root node with no action
        self.header_table = {}  # Maps actions to their first occurrence in tree

    def _update_header_table(self, node):
        """Update the header table with a new node."""
        if node.action in self.header_table:
            # Find the last node in the link list
            current = self.header_table[node.action]
            while current.next:
                current = current.next
            current.next = node
        else:
            self.header_table[node.action] = node

    def insert_action_set(self, action_set):
        """Insert a set of actions into the FP-tree."""
        current = self.root

        for action in action_set:
            if action not in current.children:
                # Create new node
                new_node = FPNode(
                    action=action,
                    parent=current
                )
                current.children[action] = new_node
                self._update_header_table(new_node)
            else:
                # Increment count of existing node
                current.children[action].count += 1

            current = current.children[action]

    def build_tree(self, pruned_action_table):
        """Build the FP-tree from the pruned action table."""
        for action_set in pruned_action_table:
            self.insert_action_set(action_set)

    def print_tree(self, node=None, level=0):
        """Print the FP-tree structure for debugging."""
        if node is None:
            node = self.root
            print("FP-Tree Structure:")

        indent = "  " * level
        action_str = str(node.action) if node.action else "Root"
        print(f"{indent}{action_str} (count: {node.count})")

        for child in node.children.values():
            self.print_tree(child, level + 1)

    def get_frequent_patterns(self, min_support=1):
        """Extract frequent patterns from the FP-tree."""
        patterns = []

        def find_prefix_paths(node):
            """Find all prefix paths for a given node."""
            paths = []
            while node and node.parent:
                if node.parent != self.root:
                    paths.append(node.parent.action)
                node = node.parent
            return paths[::-1] if paths else []

        # For each action in header table
        for action, first_node in self.header_table.items():
            pattern_base = []
            current = first_node

            # Follow node links to collect all paths
            while current:
                path = find_prefix_paths(current)
                if path:
                    pattern_base.append((path, current.count))
                current = current.next

            # If this action appears frequently enough, add it to patterns
            total_count = sum(count for _, count in pattern_base)
            if total_count >= min_support:
                patterns.append((action, total_count))

        return sorted(patterns, key=lambda x: x[1], reverse=True)

# Create and populate the FP-tree
fp_tree = FPTree()
fp_tree.build_tree(pruned_action_table)

# Print tree structure
fp_tree.print_tree()

# Get frequent patterns with minimum support of 2
frequent_patterns = fp_tree.get_frequent_patterns(min_support=7000)
print("\nFrequent Patterns:")
for pattern, support in frequent_patterns:
    print(f"Pattern: {pattern}, Support: {support}")

Streaming output truncated to the last 5000 lines.
      ('A11', 2, 0) (count: 1)
      ('A3', 'medium', 'high') (count: 1)
        ('A11', 2, 0) (count: 1)
      ('A8', 'low', 'medium') (count: 2)
        ('A11', 2, 0) (count: 1)
      ('A3', 'low', 'high') (count: 4)
        ('A8', 'medium', 'high') (count: 1)
          ('A14', 'medium', 'low') (count: 1)
            ('A11', 3, 0) (count: 1)
        ('A14', 'medium', 'low') (count: 1)
        ('A11', 2, 0) (count: 1)
        ('A11', 11, 0) (count: 1)
          ('A13', 's', 'g') (count: 1)
      ('A8', 'medium', 'high') (count: 2)
        ('A11', 2, 0) (count: 2)
    ('A2', 'medium', 'low') (count: 79)
      ('A3', 'low', 'high') (count: 7)
        ('A14', 'medium', 'low') (count: 3)
          ('A11', 2, 0) (count: 2)
          ('A11', 3, 0) (count: 1)
            ('A7', 'bb', 'v') (count: 1)
        ('A8', 'low', 'medium') (count: 1)
          ('A14', 'medium', 'low') (count: 1)
            ('A11', 2, 0) (count: 1)
        ('A8', 'me

In [25]:
def format_action_rule(rule):
    """Format an action rule for readable output."""
    actions_str = []
    for feature, old_val, new_val in rule['actions']:
        actions_str.append(f"{feature}: {old_val} → {new_val}")

    return (
        f"Rule: IF {' AND '.join(actions_str)} then A16: (-) -> (+)\n"
        f"Support: {rule['support']}\n"
        f"Confidence: {rule['confidence']:.2f}\n"
        f"{'-' * 50}"
    )

def calculate_atomic_metrics(atomic_action_sets, positive_df, negative_df):
    """
    Precompute support and confidence for all atomic actions.

    Returns:
        dict: Maps (feature, old_val, new_val) to (support, confidence)
    """
    metrics = {}

    for feat, old_val, new_val in atomic_action_sets:
        # Calculate support
        neg_count = negative_df[feat].value_counts().get(old_val, 0)
        pos_count = positive_df[feat].value_counts().get(new_val, 0)
        support = neg_count * pos_count

        # Calculate confidence
        confidence = pos_count / neg_count if neg_count > 0 else 0

        metrics[(feat, old_val, new_val)] = {
            'support': support,
            'confidence': confidence
        }

    return metrics

def extract_action_rules_with_precomputed(fp_tree, atomic_metrics, min_support=4000, min_confidence=0.1):
    """Extract action rules using precomputed metrics."""
    action_rules = []

    def traverse_path(node, current_path=None):
        if current_path is None:
            current_path = []

        if node != fp_tree.root and node.action:
            current_path.append(node.action)

            if len(current_path) > 0:
                # Get precomputed metrics for this action
                metrics = atomic_metrics[node.action]

                if metrics['support'] >= min_support and metrics['confidence'] >= min_confidence:
                    action_rules.append({
                        'actions': current_path.copy(),
                        'support': metrics['support'],
                        'confidence': metrics['confidence']
                    })

        for child in node.children.values():
            traverse_path(child, current_path.copy())

    traverse_path(fp_tree.root)
    action_rules.sort(key=lambda x: (x['support'], x['confidence']), reverse=True)
    return action_rules

# Example usage with timing:
import time

# Precompute metrics
start_time = time.time()
atomic_metrics = calculate_atomic_metrics(atomic_action_sets, positive_df, negative_df)
precompute_time = time.time() - start_time

print(f"Precomputation time: {precompute_time:.2f} seconds")

# Extract rules using precomputed metrics
start_time = time.time()
rules = extract_action_rules_with_precomputed(fp_tree, atomic_metrics, min_support=8000, min_confidence=0.8)
extraction_time = time.time() - start_time

print(f"Found {len(rules)} rules")

# Print top 5 rules
print("\nTop 5 Rules by Support and Confidence:")
for rule in rules[:5]:
    print(format_action_rule(rule))

# Print some statistics about the atomic metrics
print("\nAtomic Action Statistics:")
supports = [m['support'] for m in atomic_metrics.values()]
confidences = [m['confidence'] for m in atomic_metrics.values()]

print(f"Average Support: {sum(supports) / len(supports):.2f}")
print(f"Average Confidence: {sum(confidences) / len(confidences):.2f}")
print(f"Max Support: {max(supports)}")
print(f"Max Confidence: {max(confidences):.2f}")

Precomputation time: 28.39 seconds
Found 43179 rules

Top 5 Rules by Support and Confidence:
Rule: IF A12: t → f AND A3: high → low AND A14: high → medium AND A2: medium → low AND A5: p → g then A16: (-) -> (+)
Support: 30680
Confidence: 2.20
--------------------------------------------------
Rule: IF A12: t → f AND A3: high → low AND A14: high → medium AND A2: medium → low AND A5: p → g AND A4: y → u then A16: (-) -> (+)
Support: 30680
Confidence: 2.20
--------------------------------------------------
Rule: IF A12: t → f AND A3: high → low AND A14: high → medium AND A5: p → g then A16: (-) -> (+)
Support: 30680
Confidence: 2.20
--------------------------------------------------
Rule: IF A12: t → f AND A3: high → low AND A14: high → medium AND A5: p → g AND A4: y → u then A16: (-) -> (+)
Support: 30680
Confidence: 2.20
--------------------------------------------------
Rule: IF A12: t → f AND A3: high → low AND A2: medium → low AND A13: g → s AND A5: p → g then A16: (-) -> (+)
Support